# Heat Geodesics en NIFs

Tenemos la PDE 
$$\frac{\partial u}{\partial t} = \nabla \cdot ( P_{\nabla \psi} \nabla u) = \Delta u - (\nabla u \cdot \nabla \psi) \Delta \psi$$

Definimos $u_0(\vec{x}) = e^{-\varepsilon \left || \vec{x} - \vec{p} | \right |^2}$. Entonces,

$$\nabla u_0(\vec{x}) = -2 \varepsilon u_0(\vec{x}) (\vec{x} - \vec{p}) \quad \quad \text{y} \quad \quad \Delta u_0(\vec{x}) = 2\varepsilon \, u_0(\vec{x}) \, (2 \varepsilon \left || \vec{x} - \vec{p} | \right |^2 - 3) $$

### Forward Euler
Este esquema es mas sencillo (pero mas inestable), integramos con respecto al tiempo tomando la siguiente aproximacion:
$$u_t(\vec{x}) = u_0(\vec{x}) + t (\Delta u_0(\vec{x}) - (\nabla u_0 \cdot \nabla \psi) \Delta \psi)$$

### Backward Euler
Integrador implicito:
$$u_t = u_0 + t (\nabla \cdot \circ \, P_{\nabla \psi} \nabla) u_t$$

Donde se entiende por $(\nabla \cdot \circ \, P_{\nabla \psi}) : C^k \rightarrow C^k$ el operador que actua sobre funciones $u$, el cual aplica la proyeccion del gradiente sobre el espacio ortogonal a $\nabla \psi$ y luego la divergencia.

$$t(id - \nabla \cdot \circ \, P_{\nabla \psi} \nabla) u_t = u_0$$

Entonces, $u_t$ es la funcion que minimiza la energia:

$$ \underset{u: \Omega \rightarrow \mathbb{R}}{\text{min}} \quad \quad \int_\Omega \left | t(id - \nabla \cdot \circ \, P_{\nabla \psi} \,\nabla) u_t - u_0 \right | \quad d\vec{x}.$$

Tengo muchas formas de realizar esta optimizacion:
1. CML dada una familia de funciones
2. Discretizacion y FEM ?
3. Usando el mismo SIREN

Hay que tener fe de que estas minimizaciones ganen ventaja frente a FE... ta dificil.

In [1]:
import numpy as np
import torch
from src.model import SIREN
from src.obj import load
import trimesh as tm
import meshplot as mp
import src.diff_operators as dif

In [2]:
p = np.array( [[0.5529412, 0.2470588, 0.1302517]] )

In [ ]:
mesh = tm.load_mesh('results/bunny/experiment/reconstructions/mc_mesh_best.obj')

plot = mp.plot( mesh.vertices, mesh.faces, return_plot=True )
plot.add_points( p, shading={'point_size':0.2})

In [3]:
model = SIREN(
        n_in_features= 3,
        n_out_features=1,
        hidden_layer_config=[512]*4,
        w0=30,
        ww=None,
        activation= 'sine'
)
model.load_state_dict( torch.load('results/bunny/experiment/models/model_best.pth', weights_only=True))

device_torch = torch.device(0)
model.to(device_torch)

/home/mfainstein/miniconda3/envs/tdf/lib/python3.10/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


SIREN(
  (net): Sequential(
    (0): Sequential(
      (0): Linear(in_features=3, out_features=512, bias=True)
      (1): SineLayer(w0=30)
    )
    (1): Sequential(
      (0): Linear(in_features=512, out_features=512, bias=True)
      (1): SineLayer(w0=30)
    )
    (2): Sequential(
      (0): Linear(in_features=512, out_features=512, bias=True)
      (1): SineLayer(w0=30)
    )
    (3): Sequential(
      (0): Linear(in_features=512, out_features=512, bias=True)
      (1): SineLayer(w0=30)
    )
    (4): Sequential(
      (0): Linear(in_features=512, out_features=1, bias=True)
    )
  )
)

In [4]:
EPSILON = 0.1
pc, _ = load( 'data/bunny/bunny_pc.obj' )

u0 = lambda x, p: torch.exp( -EPSILON * torch.sum( (x - p)**2) )
grad_u0 = lambda x, p: -2 * EPSILON * u0(x, p) * (x - p)
lap_u0 = lambda x, p: 2 * EPSILON * u0(x, p) * ( 2* EPSILON * torch.sum( (x - p)**2) - 3 )

In [ ]:
#plot = mp.plot( mesh.vertices, mesh.faces, return_plot=True )
#plot.add_points( pc, shading={'point_size':0.1})

In [5]:
t = 0.001
pc = torch.from_numpy(pc).to( device_torch ).float()

In [6]:
def computeX( model, samples, p, u0, grad_u0, lap_u0, max_batch=64**2, device=torch.device(0) ):
    # samples = ( amount_samples, 3 )    
    head = 0
    amount_samples = samples.shape[0]

    uts = torch.zeros( (amount_samples, 1)).to( device )
    X = torch.zeros( (amount_samples, 3)).to( device )
    divX = torch.zeros( (amount_samples, 1)).to( device )

    ps = torch.from_numpy( np.tile(p, (max_batch, 1)) ).to(device_torch)

    while head < amount_samples:
        
        inputs_subset = samples[head:min(head + max_batch, amount_samples), :].unsqueeze(0)

        x, y =  model(inputs_subset).values()

        grad_psi = dif.gradient(y,x)
        laplace_psi = dif.divergence( grad_psi, x )

        ps_ss = ps[:inputs_subset.shape[1],:].unsqueeze(0)

        ut = u0( inputs_subset, ps_ss ) + t * ( lap_u0( inputs_subset, ps_ss ) - ( torch.sum( grad_u0( inputs_subset, ps_ss ) * grad_psi, dim=-1 ) ) * laplace_psi )
        uts[head:min(head + max_batch, amount_samples)] = ut.squeeze(0)

        X_ss = -1 * torch.nn.functional.normalize( dif.gradient( ut, x ), dim=-1 )

        X[head:min(head + max_batch, amount_samples)] = X_ss.squeeze(0)
        divX[head:min(head + max_batch, amount_samples)] = dif.divergence( X_ss, x ).squeeze(0)

        head += max_batch

    return uts, X, divX

ut, X, divX = computeX( model, pc, p, u0, grad_u0, lap_u0 )

RuntimeError: The expanded size of the tensor (1) must match the existing size (4096) at non-singleton dimension 1.  Target sizes: [4096, 1].  Tensor sizes: [4096, 4096]